In [0]:
from google.colab import drive
drive.mount('/content/drive')

In [0]:
!cp './drive/My Drive/flipkart.zip' 'images.zip'

In [0]:
!nvidia-smi

Mon Feb  4 13:30:47 2019       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 396.44                 Driver Version: 396.44                    |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   31C    P8    28W / 149W |      0MiB / 11441MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  

In [6]:
!unzip 'images.zip'

Output hidden; open in https://colab.research.google.com to view.

In [0]:
!cp './drive/My Drive/50model1.h5' 'model.h5'

In [0]:
!cp './drive/My Drive/training.csv' 'training.csv'

In [9]:
!wget https://d8it4huxumps7.cloudfront.net/uploads/attachements/code-contests/flipkartgrid/test.csv

--2019-02-04 14:14:51--  https://d8it4huxumps7.cloudfront.net/uploads/attachements/code-contests/flipkartgrid/test.csv
Resolving d8it4huxumps7.cloudfront.net (d8it4huxumps7.cloudfront.net)... 99.84.240.231, 99.84.240.121, 99.84.240.102, ...
Connecting to d8it4huxumps7.cloudfront.net (d8it4huxumps7.cloudfront.net)|99.84.240.231|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 533504 (521K) [text/csv]
Saving to: ‘test.csv’

test.csv            100%[===================>] 521.00K  --.-KB/s    in 0.04s   

2019-02-04 14:14:51 (12.5 MB/s) - ‘test.csv’ saved [533504/533504]



In [13]:
from PIL import Image
import pandas as pd
from sklearn import preprocessing as pre
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import pickle
import random

import keras
import tensorflow as tf
from keras.layers import Dense, Conv2D, BatchNormalization, Activation
from keras.layers import AveragePooling2D, MaxPooling2D, Input, Flatten
from keras.optimizers import Adam
from keras.regularizers import l2
from keras import backend as K
from keras.models import Model,load_model
from keras.callbacks import ModelCheckpoint,LearningRateScheduler,ReduceLROnPlateau
import numpy as np

plt.switch_backend('agg')

size = 480

def getdata(batch_start,batch_end):
    id_to_data={}
    id_to_size={}
    f1 = pd.read_csv('./training.csv')

    for index, row in f1.iterrows():
        if index >= batch_start and index < batch_end:
            image=Image.open("./images/"+row['image_name']).convert('RGB')
            id_to_size[index - batch_start]=np.array(image,dtype=np.float32).shape[0:2]
            image=image.resize((size,size))
            image=np.array(image,dtype=np.float32)
            #print('Image loading : ' + str(index - batch_start) + '\n')
            image=image/255
            pre.normalize(image[0])#Normalize(image,[0.485,0.456,0.406],[0.229,0.size,0.225])
            pre.normalize(image[1])
            pre.normalize(image[2])
            id_to_data[index - batch_start]=image

    id_to_data=np.array(list(id_to_data.values()))
    id_to_size=np.array(list(id_to_size.values()))

    id_to_box={}

    for index, row in f1.iterrows():
        if index >= batch_start and index < batch_end:
            box = np.array([float(i) for i in range(4)], dtype = np.float32)
            box[0]=row['x1']/id_to_size[index - batch_start][1]*size
            box[1]=row['y1']/id_to_size[index - batch_start][0]*size
            box[2]=row['x2']/id_to_size[index - batch_start][1]*size
            box[3]=row['y2']/id_to_size[index - batch_start][0]*size
            id_to_box[index - batch_start]=box

    id_to_box=np.array(list(id_to_box.values()))
    return id_to_data, id_to_box

def imageLoader(batch_size):

    L = 14000

    #this line is just to make the generator infinite, keras needs that    
    while True:

        batch_start = 0
        batch_end = batch_size

        while batch_start < L:
            limit = min(batch_end, L)
            X, Y = getdata(batch_start,batch_end)

            yield (X,Y) #a tuple with two numpy arrays with batch_size samples
            del X
            del Y
            batch_start += batch_size   
            batch_end += batch_size


def plot_model(model_details):
    fig, axs = plt.subplots(1,2,figsize=(15,5))

    axs[0].plot(range(1,len(model_details.history['my_metric'])+1),model_details.history['my_metric'])
    axs[0].plot(range(1,len(model_details.history['val_my_metric'])+1),[1.7*x for x in model_details.history['val_my_metric']])
    axs[0].set_title('Model Accuracy')
    axs[0].set_ylabel('Accuracy')
    axs[0].set_xlabel('Epoch')
    axs[0].set_xticks(np.arange(1,len(model_details.history['my_metric'])+1),len(model_details.history['my_metric'])/10)
    axs[0].legend(['train', 'val'], loc='best')

    axs[1].plot(range(1,len(model_details.history['loss'])+1),model_details.history['loss'])
    axs[1].plot(range(1,len(model_details.history['val_loss'])+1),model_details.history['val_loss'])
    axs[1].set_title('Model Loss')
    axs[1].set_ylabel('Loss')
    axs[1].set_xlabel('Epoch')
    axs[1].set_xticks(np.arange(1,len(model_details.history['loss'])+1),len(model_details.history['loss'])/10)
    axs[1].legend(['train', 'val'], loc='best')

    plt.savefig("model1.png")



# metric function
def my_metric(labels,predictions):
    threshhold=0.75
    x=predictions[:,0]*size
    x=tf.maximum(tf.minimum(x,size),0.0)
    y=predictions[:,1]*size
    y=tf.maximum(tf.minimum(y,size),0.0)
    width=predictions[:,2]*size
    width=tf.maximum(tf.minimum(width,size),0.0)
    height=predictions[:,3]*size
    height=tf.maximum(tf.minimum(height,size),0.0)
    label_x=labels[:,0]
    label_y=labels[:,1]
    label_width=labels[:,2]
    label_height=labels[:,3]
    a1=tf.multiply(width,height)
    a2=tf.multiply(label_width,label_height)
    x1=tf.maximum(x,label_x)
    y1=tf.maximum(y,label_y)
    x2=tf.minimum(x+width,label_x+label_width)
    y2=tf.minimum(y+height,label_y+label_height)
    IoU=tf.abs(tf.multiply((x1-x2),(y1-y2)))/(a1+a2-tf.abs(tf.multiply((x1-x2),(y1-y2))))
    #condition=tf.less(threshhold,IoU)
    #sum=tf.where(condition,tf.ones(tf.shape(condition)),tf.zeros(tf.shape(condition)))
    #return tf.reduce_mean(sum)
    return IoU

# loss function
def smooth_l1_loss(true_box,pred_box):
    loss=0.0
    for i in range(4):
        residual=tf.abs(true_box[:,i]-pred_box[:,i]*size)
        condition=tf.less(residual,1.0)
        small_res=0.5*tf.square(residual)
        large_res=residual-0.5
        loss=loss+tf.where(condition,small_res,large_res)
    return tf.reduce_mean(loss)


def resnet_block(inputs,num_filters,kernel_size,strides,activation='relu'):
    x=Conv2D(num_filters,kernel_size=kernel_size,strides=strides,padding='same',kernel_initializer='he_normal',kernel_regularizer=l2(1e-3))(inputs)
    x=BatchNormalization()(x)
    if(activation):
        x=Activation('relu')(x)
    return x


def resnet18():
    inputs=Input((size,size,3))
    
    # conv1
    x=resnet_block(inputs,64,[7,7],2)

    # conv2
    x=MaxPooling2D([3,3],2,'same')(x)
    for i in range(2):
        a=resnet_block(x,64,[3,3],1)
        b=resnet_block(a,64,[3,3],1,activation=None)
        x=keras.layers.add([x,b])
        x=Activation('relu')(x)
    
    # conv3
    a=resnet_block(x,128,[1,1],2)
    b=resnet_block(a,128,[3,3],1,activation=None)
    x=Conv2D(128,kernel_size=[1,1],strides=2,padding='same',kernel_initializer='he_normal',kernel_regularizer=l2(1e-3))(x)
    x=keras.layers.add([x,b])
    x=Activation('relu')(x)

    a=resnet_block(x,128,[3,3],1)
    b=resnet_block(a,128,[3,3],1,activation=None)
    x=keras.layers.add([x,b])
    x=Activation('relu')(x)

    # conv4
    a=resnet_block(x,256,[1,1],2)
    b=resnet_block(a,256,[3,3],1,activation=None)
    x=Conv2D(256,kernel_size=[1,1],strides=2,padding='same',kernel_initializer='he_normal',kernel_regularizer=l2(1e-3))(x)
    x=keras.layers.add([x,b])
    x=Activation('relu')(x)

    a=resnet_block(x,256,[3,3],1)
    b=resnet_block(a,256,[3,3],1,activation=None)
    x=keras.layers.add([x,b])
    x=Activation('relu')(x)

    # conv5
    a=resnet_block(x,512,[1,1],2)
    b=resnet_block(a,512,[3,3],1,activation=None)
    x=Conv2D(512,kernel_size=[1,1],strides=2,padding='same',kernel_initializer='he_normal',kernel_regularizer=l2(1e-3))(x)
    x=keras.layers.add([x,b])
    x=Activation('relu')(x)

    a=resnet_block(x,512,[3,3],1)
    b=resnet_block(a,512,[3,3],1,activation=None)
    x=keras.layers.add([x,b])
    x=Activation('relu')(x)

    x=AveragePooling2D(pool_size=7,data_format="channels_last")(x)
    # out:1*1*512

    y=Flatten()(x)
    # out:512
    y=Dense(1000,kernel_initializer='he_normal',kernel_regularizer=l2(1e-3))(y)
    outputs=Dense(4,kernel_initializer='he_normal',kernel_regularizer=l2(1e-3))(y)
    
    model=Model(inputs=inputs,outputs=outputs)
    return model
'''
model = resnet18()


#model.compile(loss="smooth_l1_loss",optimizer=Adam(),metrics=["my_metric"])
model.compile(loss=smooth_l1_loss,optimizer=Adam(), metrics=[my_metric])

model.summary()
'''

print("Loading model \n")
model = load_model('50model1.h5', custom_objects={'smooth_l1_loss': smooth_l1_loss, 'my_metric': my_metric}, compile = False)
model.summary()
print("Loaded model \n")


model.compile(loss=smooth_l1_loss,optimizer=Adam(), metrics=[my_metric])

def lr_sch(epoch):
    #200 total
    return 1e-5
    '''if epoch <25:
        return 1e-3
    if 25<=epoch<50:
        return 1e-4
    if epoch>=50:
        return 1e-5'''

lr_scheduler=LearningRateScheduler(lr_sch)
lr_reducer=ReduceLROnPlateau(monitor='val_my_metric',factor=0.2,patience=5,mode='max',min_lr=1e-3)

checkpoint=ModelCheckpoint('50model1.h5',monitor='val_loss',verbose=0,save_best_only=True,mode='auto')

model_details=model.fit_generator(imageLoader(64),steps_per_epoch=110,epochs=30, callbacks=[lr_scheduler,lr_reducer,checkpoint], validation_data=imageLoader(64),validation_steps=11,shuffle=True)

model.save('50model1.h5')

plot_model(model_details)


Loading model 

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_4 (InputLayer)            (None, 480, 480, 3)  0                                            
__________________________________________________________________________________________________
conv2d_61 (Conv2D)              (None, 240, 240, 64) 9472        input_4[0][0]                    
__________________________________________________________________________________________________
batch_normalization_52 (BatchNo (None, 240, 240, 64) 256         conv2d_61[0][0]                  
__________________________________________________________________________________________________
activation_52 (Activation)      (None, 240, 240, 64) 0           batch_normalization_52[0][0]     
_____________________________________________________________________________________________

In [15]:
#Testing

import pickle
import numpy as np
import matplotlib.pyplot as plt
import keras
from keras.models import load_model
from sklearn import preprocessing as pre
import random
import math

from PIL import Image
import pandas as pd
import keras
import tensorflow as tf
from keras.layers import Dense, Conv2D, BatchNormalization, Activation
from keras.layers import AveragePooling2D, MaxPooling2D, Input, Flatten
from keras.optimizers import Adam
from keras.regularizers import l2
from keras import backend as K
from keras.models import Model,load_model
from keras.callbacks import ModelCheckpoint,LearningRateScheduler,ReduceLROnPlateau
import numpy as np
size = 480
def my_metric(labels,predictions):
    threshhold=0.75
    x=predictions[:,0]*size
    x=tf.maximum(tf.minimum(x,size),0.0)
    y=predictions[:,1]*size
    y=tf.maximum(tf.minimum(y,size),0.0)
    width=predictions[:,2]*size
    width=tf.maximum(tf.minimum(width,size),0.0)
    height=predictions[:,3]*size
    height=tf.maximum(tf.minimum(height,size),0.0)
    label_x=labels[:,0]
    label_y=labels[:,1]
    label_width=labels[:,2]
    label_height=labels[:,3]
    a1=tf.multiply(width,height)
    a2=tf.multiply(label_width,label_height)
    x1=tf.maximum(x,label_x)
    y1=tf.maximum(y,label_y)
    x2=tf.minimum(x+width,label_x+label_width)
    y2=tf.minimum(y+height,label_y+label_height)
    IoU=tf.abs(tf.multiply((x1-x2),(y1-y2)))/(a1+a2-tf.abs(tf.multiply((x1-x2),(y1-y2))))
    condition=tf.less(threshhold,IoU)
    sum=tf.where(condition,tf.ones(tf.shape(condition)),tf.zeros(tf.shape(condition)))
    return tf.reduce_mean(sum)

# loss function
def smooth_l1_loss(true_box,pred_box):
    loss=0.0
    for i in range(4):
        residual=tf.abs(true_box[:,i]-pred_box[:,i]*size)
        condition=tf.less(residual,1.0)
        small_res=0.5*tf.square(residual)
        large_res=residual-0.5
        loss=loss+tf.where(condition,small_res,large_res)
    return tf.reduce_mean(loss)

plt.switch_backend('agg')
model = load_model('50model2.h5', custom_objects={'smooth_l1_loss': smooth_l1_loss, 'my_metric': my_metric})
csv={}
f1 = pd.read_csv('./test.csv')
i = 0
id_to_data={}
for index, row in f1.iterrows():
    print("Predicting "+str(i)+"th image.")
    image=Image.open("./images/"+row['image_name']).convert('RGB')
    id_to_size=np.array(image,dtype=np.float32).shape[0:2]
    image=image.resize((size,size))
    image=np.array(image,dtype=np.float32)
    image=image/255
    pre.normalize(image[0])
    pre.normalize(image[1])
    pre.normalize(image[2])
    id_to_data={}
    id_to_data[0]=image
    id_to_data=np.array(list(id_to_data.values()))
    result1=model.predict(id_to_data)
    del id_to_data
    result = result1[0]
    image=image*255
    image=image.astype(np.uint8)
    #plt.imshow(image)
    box = np.array([float(i) for i in range(5)], dtype = object)
    box[0]=row['image_name']
    box[1]=int(round((result[0] if result[0] > 0  else 0)*(640)))
    box[2]=int(round((result[2] if result[2] > 0  else 0)*(640)))
    box[3]=int(round((result[1] if result[1] > 0  else 0)*(480)))
    box[4]=int(round((result[3] if result[3] > 0  else 0)*(480)))
    csv[i]=box
    i = i + 1
    #plt.gca().add_patch(plt.Rectangle((result[0]*size,result[1]*size),(result[2] - result[0])*size,(result[3] - result[1])*size,fill=False,edgecolor='green',linewidth=2,alpha=0.5))
    #plt.show()
    #plt.savefig(str(i)+".png")
    #plt.cla()
csv=np.array(list(csv.values()))
fmt='%s, %d, %d, %d, %d'
np.savetxt("test_result.csv", csv,fmt=fmt,  header="image_name,x1,x2,y1,y2",delimiter=",")

Predicting 0th image.
Predicting 1th image.
Predicting 2th image.
Predicting 3th image.
Predicting 4th image.
Predicting 5th image.
Predicting 6th image.
Predicting 7th image.
Predicting 8th image.
Predicting 9th image.
Predicting 10th image.
Predicting 11th image.
Predicting 12th image.
Predicting 13th image.
Predicting 14th image.
Predicting 15th image.
Predicting 16th image.
Predicting 17th image.
Predicting 18th image.
Predicting 19th image.
Predicting 20th image.
Predicting 21th image.
Predicting 22th image.
Predicting 23th image.
Predicting 24th image.
Predicting 25th image.
Predicting 26th image.
Predicting 27th image.
Predicting 28th image.
Predicting 29th image.
Predicting 30th image.
Predicting 31th image.
Predicting 32th image.
Predicting 33th image.
Predicting 34th image.
Predicting 35th image.
Predicting 36th image.
Predicting 37th image.
Predicting 38th image.
Predicting 39th image.
Predicting 40th image.
Predicting 41th image.
Predicting 42th image.
Predicting 43th image